In [ ]:
from IPython.display import display
import pandas as pd
import csv
import random
import re
# import pyperclip
import numpy as np
import seaborn as sns
import requests
import matplotlib, matplotlib.pyplot as plt
import pprint
import os
import dateutil
import datetime

In [ ]:
def to_number(cell):
    if not isinstance(cell, str):
        return cell
    _ = re.findall(r'\d+', cell)
    if _:
        return int(_[0])


df = pd.read_excel("赛题1数据材料/赛题1数据集/购地-地块公示.xlsx", index_col="企业编号")
df['出让年限'] = df['出让年限'].map(to_number)
df

In [ ]:
def to_day(cell):
    if not isinstance(cell, str):
        return cell
    return (dateutil.parser.parse(cell) - datetime.datetime(2019, 3, 5)).days

In [ ]:
time = df[['时间']].applymap(to_day).groupby('企业编号').mean().T
time = time.reindex(columns=list(range(1001, 4001)))
time

In [ ]:
lst = df['土地用途'].value_counts()[df['土地用途'].value_counts() == 1].index


def to_land_use(cell):
    if not isinstance(cell, str):
        return cell
    if cell not in lst:
        return cell
    
    
land_use = pd.get_dummies(df['土地用途'].map(to_land_use), prefix='土地用途').groupby('企业编号').sum().T
land_use = land_use.reindex(columns=list(range(1001, 4001))).fillna(0)
land_use

In [ ]:
def shaixuan(cell):
    if(cell)>30:
        return np.nan
    else:
        return cell

#  把面积过大的企业置nan

In [ ]:
land_area_hectare = df[['土地面积（公顷）']].groupby('企业编号').sum().T
land_area_hectare = land_area_hectare.reindex(columns=list(range(1001, 4001)))
land_area_hectare=land_area_hectare.applymap(shaixuan)
land_area_hectare

In [ ]:
def nianxian(cell):
    if cell>70:
        return np.nan
    return cell

In [ ]:
transfer_period = df[['出让年限']].groupby('企业编号').mean().T
transfer_period = transfer_period.reindex(columns=list(range(1001, 4001))).applymap(nianxian)
transfer_period

In [ ]:
#transaction_price_ten_thousand_yuan = df[['成交价（万元）']].groupby('企业编号').sum().T
#transaction_price_ten_thousand_yuan = transaction_price_ten_thousand_yuan.reindex(columns=list(range(1001, 4001)))
#transaction_price_ten_thousand_yuan

In [ ]:
pd.concat([time, land_use, land_area_hectare, transfer_period]).to_pickle('购地-地块公示.pickle')